In [1]:
# Load dependencies
import binascii
import json
import pandas as pd
import requests
import struct
import time
import warnings

from collections import OrderedDict
from operator    import getitem
from web3        import Web3

In [2]:
# Contract information
beacon_chain_contract      = Web3.to_checksum_address('0x00000000219ab540356cbb839cbe05303d7705fa')
beacon_chain_deposit_event = '0x649bbc62d0e31342afea4e5cd82d4049e7e1ee912fc0889aa790803be39038c5'

In [3]:
# API Information
api_key = '<API key goes here>'
headers = {
    "accept"                    : "application/json",
    "x-amberdata-blockchain-id" : "ethereum-mainnet",
    "x-api-key"                 : f'{api_key}'
}

# RPC URL
eth_node_url = f'https://rpc.web3api.io/api/v2?x-api-key={api_key}'

# Contract URL
abi_url = f'https://web3api.io/api/v2/contracts/{beacon_chain_contract}' 

In [4]:
# Web3 Client
w3 = Web3(Web3.HTTPProvider(eth_node_url))

# Check if connected to Ethereum node
if w3.is_connected:
    print("Connected to Ethereum node")
else:
    print("Not connected to Ethereum node")
    exit(1)

Connected to Ethereum node


In [5]:
# Fetching contract address ABI, formating and extracting ABI Only. Will use later for parsing logs. 
abi = requests.request("GET", url=abi_url, headers=headers)
abi = json.loads(abi.text)
abi = abi['payload']['abi']

In [6]:
abi

[{'inputs': [], 'stateMutability': 'nonpayable', 'type': 'constructor'},
 {'anonymous': False,
  'inputs': [{'indexed': False,
    'internalType': 'bytes',
    'name': 'pubkey',
    'type': 'bytes'},
   {'indexed': False,
    'internalType': 'bytes',
    'name': 'withdrawal_credentials',
    'type': 'bytes'},
   {'indexed': False,
    'internalType': 'bytes',
    'name': 'amount',
    'type': 'bytes'},
   {'indexed': False,
    'internalType': 'bytes',
    'name': 'signature',
    'type': 'bytes'},
   {'indexed': False,
    'internalType': 'bytes',
    'name': 'index',
    'type': 'bytes'}],
  'name': 'DepositEvent',
  'type': 'event'},
 {'inputs': [{'internalType': 'bytes', 'name': 'pubkey', 'type': 'bytes'},
   {'internalType': 'bytes',
    'name': 'withdrawal_credentials',
    'type': 'bytes'},
   {'internalType': 'bytes', 'name': 'signature', 'type': 'bytes'},
   {'internalType': 'bytes32',
    'name': 'deposit_data_root',
    'type': 'bytes32'}],
  'name': 'deposit',
  'outputs': 

In [7]:
# Create Web3 Contract Instance: 
myContract = w3.eth.contract(address=beacon_chain_contract, abi=abi)

In [8]:
# Helper function to convert bytes to interger. 
def endian_to_int(endian):
    little_endian_bytes = bytes.fromhex(endian)
    reversed_bytes      = little_endian_bytes[::-1]
    integer_value       = int.from_bytes(reversed_bytes, byteorder='big')
    return integer_value

In [9]:
# Empty dictionary for storing returned data. 
collection = {}
id         = 0

# Pagination information
pagination          =    0
total_pages         = 1000
paginationIncrement =    1

# Loop through the pages
while pagination <= total_pages:
    # Fetch Contract Events Logs. 
    url = f'https://web3api.io/api/v2/addresses/{beacon_chain_contract}/logs?topic={beacon_chain_deposit_event}&page={pagination}&size=100'

    response = requests.request("GET", url, headers=headers)
    while response.status_code == 429:
        time.sleep(1)
        response = requests.request("GET", url, params=querystring)

    data = json.loads(response.text)

    if not "payload" in data:
        print('No data for page', pagination)
        continue;

    batch_list = data["payload"]['records']
    for item in batch_list:
            try: 
                id += 1
                collection[id]                           = {}
                collection[id]['timestamp']              = item["timestamp"]
                collection[id]['transaction_hash']       = item["transactionHash"]
                collection[id]['block_number']           = item['blockNumber']
                collection[id]['pub_key']                = item['data'][6] + item['data'][7][:32]
                collection[id]['withdrawal_credentials'] = item['data'][9]
                collection[id]['amount']                 = endian_to_int(item['data'][11])
                collection[id]['deposit_index']          = endian_to_int(item['data'][17])
                collection[id]['signature']              = item['data'][13] + item['data'][14] + item['data'][15]
            except NameError:
                print(NameError)
            except:
                print("No Data")

    print('Finished page', pagination, ', count =', id)
    pagination += paginationIncrement

Finished page 0 , count = 100
Finished page 1 , count = 200
Finished page 2 , count = 300
Finished page 3 , count = 400
Finished page 4 , count = 500
Finished page 5 , count = 600
Finished page 6 , count = 700
Finished page 7 , count = 800
Finished page 8 , count = 900
Finished page 9 , count = 1000
Finished page 10 , count = 1100
Finished page 11 , count = 1200
Finished page 12 , count = 1300
Finished page 13 , count = 1400
Finished page 14 , count = 1500
Finished page 15 , count = 1600
Finished page 16 , count = 1700
Finished page 17 , count = 1800
Finished page 18 , count = 1900
Finished page 19 , count = 2000
Finished page 20 , count = 2100
Finished page 21 , count = 2200
Finished page 22 , count = 2300
Finished page 23 , count = 2400
Finished page 24 , count = 2500
Finished page 25 , count = 2600
Finished page 26 , count = 2700
Finished page 27 , count = 2800
Finished page 28 , count = 2900
Finished page 29 , count = 3000
Finished page 30 , count = 3100
Finished page 31 , count = 

In [10]:
# Updating some pandas default settings
pd.set_option('max_colwidth',         None)                 # Show full width of showing columns
pd.set_option("expand_frame_repr",    False)                # Print columns
pd.set_option('display.float_format', lambda x: f'{x:.3f}') # Format number

# Load data in data frame
df = pd.DataFrame.from_dict(collection, orient='index')

In [11]:
# Parse timestamps
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')

In [12]:
# Display data
df

,timestamp,transaction_hash,block_number,pub_key,withdrawal_credentials,amount,deposit_index,signature,datetime
1,1736928191000,0x36b8798fab4b1d45bd3b32c4bc16cf2427156b8e7994b5d8b664863e997b8cb9,21628564,8633406a6d02d6cd23289d2473384d8dad8dd49227f523c24e90c8655bc8be7abff1c9b97089815bf9ade53d2e7d640b,0100000000000000000000002b78035514401ed1592eb691b8673a93edf97470,32000000000,1867640,849c899e0a5a22eb92420ddf31d06778c9957427753870e230c48850f7105a03bf2d681cd223d24398acf868eaae651e1606f191d9e1e622166ebd4f88dccc21d2eded50d53f49e21e51c2b64e0057df3f4e24efdb29eac9cf907e30eca2de9b,2025-01-15 08:03:11
2,1736927675000,0x3fc4d7cc5f4aa0a30bf7270d20d04bccb92ca899bfd536e4d236fc7cf2894e37,21628522,ab16f54cd0932544af6208760bfc1d67d8792a664e27ef5811967a12465636e8d9bfc412c8d61e8089c1198bf6831301,0100000000000000000000002b78035514401ed1592eb691b8673a93edf97470,32000000000,1867639,a36d40d682e27b00041ac5ae81e91d4af8c985983c9448f0b9d16b61dc91e38871ec69b858f969c5898ee13eb06862fa14eb46b6471108672a8c8a92abe8c32df4ceb9f755c5dac1fc853d620b2ddf4b1fef4a4fdd9d0053de695ae0df7dc6a2,2025-01-15 07:54:35
3,1736927603000,0xb0c1df735022ded868294e5ebded41419b2292623cc618b223fc65edd2303218,21628516,9572a01763440c19014fde77cd6f964a873b3a0a95d8cea859d51cece5f398ac97ede52a610fc318831595dcd5390f01,0100000000000000000000002b78035514401ed1592eb691b8673a93edf97470,32000000000,1867638,a649f027ae5afcbf6a0adad9b3d3f62a0310321260e9c359e38dd4d65457ed9d43e3a50e11734b57db1351aad4c5483c187bb57ef14415902f2a67f520c59d6f302ba6e2e0e09705996369241af7148c0180e96c4f78b10dedb727c31a55f27d,2025-01-15 07:53:23
4,1736927435000,0xd577d2a2626216a9f182fa2d8f581e90a6dffc9cab52f5c847d034b410bb78b5,21628502,b2ce6a730aa98a79e4ae3054535144aa910992938b236e67e56587d524fe923de7bf5e4a4cb6c345ea58c7bef87701a4,0100000000000000000000002b78035514401ed1592eb691b8673a93edf97470,32000000000,1867637,a4099400290df189c366b2a51c95f07184a347a49b36e980e2d69dafbddb845bc1608a2507408afc76281ae755d12bec0cbc8397f22e31924c6adef60dd0f064b18b134486c8105bcb08a30fd54a65a2f180eeaad09a4f5782aa657408d4a9c8,2025-01-15 07:50:35
5,1736924723000,0x4fc3b5ff0dd88a1c2d1a40b5a9afc0e52641ed8d8df68c92917ff908618183b4,21628277,b0a960f70304dd995526f6fc9225cff0349e893550c4bb45382005a770408b1a5cdf9ebe44ce1623e8d8597acf40fa24,0100000000000000000000006be457e04092b28865e0cba84e3b2cfa0f871e67,32000000000,1867636,80c5a2fe989d875266a5092bc7a7b7a486d36a03418d8ab9072da3e3ad20fcbd49b5d786db95b7b9895882a4331614fd00c2746e5aa222515ce578f37b7f7ab81fa6878ba6e9dc751c04ec6217a3b6cc4397c550703b6c618e71d60ee16a55f0,2025-01-15 07:05:23
...,...,...,...,...,...,...,...,...,...
100096,1731507023000,0x93604d26fddcdad3a91cd930f8ff4c4edb2113e6f3ed19f19e7f504b3888bc4e,21179280,af24b17c518e45d3bb3de3712b2a294d8aa2373958c1b14b670be39189569ddd137e57418c8fb4351798b26c0894b2e9,0100000000000000000000000f3a1bfd2a873c36bae8a7d442247fe4b8b88a69,32000000000,1767551,8caf263e6c787456fb33085cee4f5d3f7831428d433cfb2b25467f82e697372960b34cd416c9bbd9aaed50f2623e017c0c0b15ff55762e53b5678ce200b764305c6a058fc72c83bb8be59b47373b929612c2b086ec844fe44bef518231ecec87,2024-11-13 14:10:23
100097,1731507023000,0x93604d26fddcdad3a91cd930f8ff4c4edb2113e6f3ed19f19e7f504b3888bc4e,21179280,804080b3cb850dea54f104e4016a5d395853ea464e81bdd7539419bd448616b54c9c80eb04766e63166291162ec4624c,0100000000000000000000000f3a1bfd2a873c36bae8a7d442247fe4b8b88a69,32000000000,1767550,a5b41fca3a8afd8a2b0b222088bb1f0ab4803ef1df68316eef1db174d9cc8aa232c107fc33cfe63a1da9a8d09906949d0efa59ffa3a19faaaf4562d932bd35f23a78139d64fce5506eb01a818ea5d1c363465abeb69def70237fc393d9ffc3d1,2024-11-13 14:10:23
100098,1731507023000,0x93604d26fddcdad3a91cd930f8ff4c4edb2113e6f3ed19f19e7f504b3888bc4e,21179280,945f553a891a6f012cb4cca0ec298c91205ab71a0e629d7bef118b4ce137674a94503b1f51c1fb06394317c377ecc62c,0100000000000000000000000f3a1bfd2a873c36bae8a7d442247fe4b8b88a69,32000000000,1767549,b716eeff92d48e70380bfece4b92d4f92dd24c447b9ce154137a05ab44a96182af7d99363a31c638cbe728198e0fb86e0f98e8a352e750a1d361ec6a116d460bba32780ab023a5dfe0c9a5

In [13]:
#Alternative Approach is using the ABI + Web3.py to Process_Receipt - This approach is a bit slower since it request an RPC call. 
# while pagination <= total_pages:
#     #Fetch Contract Events Logs. 
#     url = f'https://web3api.io/api/v2/addresses/{beacon_chain_contract}/logs?topic={beacon_chain_deposit_event}&page={pagination}&size=100'
#     response = requests.request("GET", url, headers=headers)
#     while response.status_code == 429:
#         time.sleep(1)
#         response = requests.request("GET", url, params=querystring)
#     batch_list = json.loads(response.text)["payload"]['records']
#     for item in batch_list:
#         try: 
#             print(item["transactionHash"])
#             receipt = w3.eth.get_transaction_receipt(item["transactionHash"])
#             logs = myContract.events.DepositEvent().process_receipt(receipt)
#             id +=1
#             collection[id] = {}
#             collection[id]['timestamp'] = item["timestamp"]
#             collection[id]['transaction_hash'] = item["transactionHash"]
#             collection[id]['blockNumber'] = logs[0]['blockNumber']
#             collection[id]['amount_raw'] = logs[0]['args']['amount']
#             collection[id]['amount_clean'] = endian_to_int(logs[0]['args']['amount'])
#             collection[id]['index'] = logs[0]['args']['index']
#             collection[id]['signature'] = logs[0]['args']['signature']
#             collection[id]['signature_cleaned'] = binascii.hexlify(logs[0]['args']['signature']).decode('utf-8')
#             collection[id]['withdrawal_credentials'] = logs[0]['args']['withdrawal_credentials']
#             collection[id]['withdrawal_credentials_cleaned'] = binascii.hexlify(logs[0]['args']['withdrawal_credentials']).decode('utf-8')
#         except NameError:
#             print(NameError)
#         except:
#             print("No Events at this TX")
#     print('Finished page',pagination)
#     pagination += paginationIncrement
